In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Lambda

In [ ]:
from keras.layers.convolutional import Conv2D, Cropping2D

In [ ]:
import numpy as np
import pandas as pd
import cv2

In [ ]:
from glob import glob
driving_logs = glob("driving_log*.csv")

In [ ]:
logs_df = [pd.read_csv(log, header=None) for log in driving_logs]

In [ ]:
driving_log = pd.concat(logs_df)
driving_log.iloc[:, 0] = driving_log.iloc[:,0].str.replace("\\", "/").str.split("/").str[-1]
features = driving_log.iloc[:, [0, 3]]

In [ ]:
X, y = features.iloc[:, 0].values, features.iloc[:, 1].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.utils import shuffle

def datagen(X, y, batch_size=4):
    num_samples = X.shape[0]
    while 1:
        shuffle(X, y)
        for offset in range(0, num_samples, batch_size):
            files = X[offset:offset+batch_size]
            angles = y[offset:offset+batch_size]
            image_batch = []
            angle_batch = []
            for f, a in zip(files, angles):
                image = cv2.imread('IMG/' + f)
                image_batch.append(image)
                angle_batch.append(float(a))
            yield shuffle(np.array(image_batch), np.array(angle_batch))
train_gen = datagen(X_train, y_train)
valid_gen = datagen(X_test, y_test)

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((60, 20), (0, 0))))
model.add(Conv2D(24, (5,5), activation='relu'))
model.add(Dropout(0.5))
#model.add(Conv2D(36, (5,5), activation='relu'))
#model.add(Conv2D(48, (5,5), activation='relu'))
#model.add(Conv2D(64, (3,3), activation='relu'))
#model.add(Conv2D(64, (3,3), activation='relu'))
#model.add(Dropout(0.5))
model.add(Flatten())
#model.add(Dense(1164))
#model.add(Dense(100))
#model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
hist = model.fit_generator(train_gen, steps_per_epoch=X_train.shape[0]//32, verbose=1, validation_data = valid_gen, validation_steps=2, epochs=3)

In [ ]:
model.save('model.h5')

In [ ]:
train_batch = next(train_gen)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
plt.imshow(train_batch[0][0])

In [ ]:
train_batch